In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 4.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=94e131762fc1f610d50d5c1ff3158b4c2822b598a38b1d347cbcfdbc1af99dc0
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=bd8841e3ca651a047ada6276764d1259ad3626afa7bf4bc7c954311d401e2051
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [27]:
import tensorflow as tf
from tensorflow import keras
import numpy as n

In [28]:
fashion_mnist=keras.datasets.fashion_mnist

In [29]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [30]:
train_images=train_images/255.0
test_images=test_images/255.0

In [31]:
train_images[0].shape

(28, 28)

In [32]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [33]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
        activation='relu'  
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')#output layer
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model


In [34]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [40]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

In [38]:
model=tuner_search.get_best_models(num_models=1)[0]

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 48)        38448     
_________________________________________________________________
flatten (Flatten)            (None, 19200)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                1536080   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 1,576,170
Trainable params: 1,576,170
Non-trainable params: 0
_________________________________________________________________
